In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("creditcardcli.csv",header=None)

In [3]:
df = df.drop([0])

In [4]:
df.columns = df.iloc[0]
df = df.iloc[1:]
df.drop(df.columns[0], axis=1, inplace=True)
df.columns.name = "ID"
df.reset_index(drop=True, inplace=True)
df

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [5]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object', name='ID')

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [113]:
X = df.drop(columns=['default payment next month'])
y = df['default payment next month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# para usar LSTM se ajusta:
num_features = X_train.shape[1]
T = None 

for i in range(1, num_features + 1):
    if num_features % i == 0:
        T = i
        break

F = num_features // T

X_train = X_train.reshape(X_train.shape[0], T, F)
X_test = X_test.reshape(X_test.shape[0], T, F)
# fin ajuste

# para sequential
# X_train = X_train.reshape(X_train.shape[0], 10, -1)
# X_test = X_test.reshape(X_test.shape[0], 10, -1)


y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


In [114]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ]) # 82.15

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='leaky_relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(128, activation='leaky_relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(64, activation='leaky_relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(32, activation='leaky_relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ]) # 82.22 con relu, 

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ]) # 81.88

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ]) # 81.88

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation='tanh', input_shape=(X_train.shape[1],)),  # Using tanh
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(64, activation='tanh'),  # Using tanh
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(32, activation='tanh'),  # Using tanh
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ]) # 82.15


# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])


In [115]:
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

In [141]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2, input_shape=(T, F)),
    tf.keras.layers.LSTM(64, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])




# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
375/375 [==============================] - 5s 7ms/step - loss: 0.4918 - accuracy: 0.8033 - val_loss: 0.4447 - val_accuracy: 0.8125
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4498 - accuracy: 0.8113 - val_loss: 0.4374 - val_accuracy: 0.8157
Epoch 3/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4474 - accuracy: 0.8127 - val_loss: 0.4394 - val_accuracy: 0.8127
Epoch 4/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4456 - accuracy: 0.8138 - val_loss: 0.4366 - val_accuracy: 0.8148
Epoch 5/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4436 - accuracy: 0.8125 - val_loss: 0.4378 - val_accuracy: 0.8158
Epoch 6/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4435 - accuracy: 0.8137 - val_loss: 0.4353 - val_accuracy: 0.8150
Epoch 7/50
375/375 [==============================] - 2s 5ms/step - loss: 0.4417 - accuracy: 0.8159 - val_loss: 0.4352 - val_accuracy: 0.8138
Epoch 

In [142]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# prediccion usando modelo
predictions = model.predict(X_test)

188/188 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8155
Test Loss: [0.43280288577079773, 0.815500020980835]
188/188 [==============================] - 1s 1ms/step


In [143]:
predictions

array([[0.1660404 ],
       [0.07815943],
       [0.12524506],
       ...,
       [0.1835067 ],
       [0.16528858],
       [0.10389739]], dtype=float32)

In [144]:
from sklearn.metrics import accuracy_score

In [145]:
import numpy as np

element_list = [item[0] for item in predictions]

binary_predictions = np.round(element_list)

accuracy = np.mean(binary_predictions == y_test)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 81.55%
